# MLP with TensorFlow Keras

In [92]:
import tensorflow as tf # type: ignore
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import json


In [93]:
data = pd.read_csv("../data/iris.csv")

In [94]:
df_x = data.iloc[:, 1:5]
df_y = data.iloc[:, 5]

In [97]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Encode target labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df_y)

# Convert integers to one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(df_x, onehot_encoded, test_size=0.2, random_state=42)


In [99]:
with open('../models/mlp.json', 'r') as f:
    json_data = json.load(f)

# Extract data from JSON
input_data = np.array(json_data['case']['input'])
target_data = np.array(json_data['case']['target'])
initial_weights = [np.array(layer) for layer in json_data['case']['initial_weights']]

# Define the model architecture
model = tf.keras.Sequential()
for i, layer in enumerate(json_data['case']['model']['layers']):
    if i == 0:
        model.add(tf.keras.layers.Dense(layer['number_of_neurons'], use_bias=True, activation=layer['activation_function'], input_shape=(json_data['case']['model']['input_size'],)))
    else:
        model.add(tf.keras.layers.Dense(layer['number_of_neurons'], activation=layer['activation_function']))

# Set initial weights
for i, layer in enumerate(model.layers):
    weights = initial_weights[i][1:]
    biases = initial_weights[i][0]
    layer.set_weights([weights, biases])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=json_data['case']['learning_parameters']['learning_rate']),
              loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(input_data, target_data, epochs=json_data['case']['learning_parameters']['max_iteration'], batch_size=json_data['case']['learning_parameters']['batch_size'])

# Evaluate the model
final_weights = [layer.get_weights() for layer in model.layers]

# Check if final weights match the expected values
# Print final weights obtained from the model
print("Final weights:")
for i, (weights, biases) in enumerate(final_weights):
    print(f"Layer {i + 1}:")
    print("Weights:")
    print(weights)
    print("Biases:")
    print(biases)

# Print expected final weights from the JSON data
print("\nExpected final weights:")
for i, layer_weights in enumerate(json_data['expect']['final_weights']):
    print(f"Layer {i + 1}:")
    print("Weights:")
    print(layer_weights)

# Evaluate the model
test_loss = model.evaluate(input_data, target_data, verbose=0)
print("Test Loss:", test_loss)

# Calculate accuracy
predictions = model.predict(input_data, verbose=0)
accuracy = np.mean(np.equal(np.argmax(target_data, axis=1), np.argmax(predictions, axis=1)))
print("Accuracy:", accuracy)

/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 816ms/step - accuracy: 1.0000 - loss: 0.3385
Final weights:
Layer 1:
Weights:
[[-0.3999977   0.40000233]
 [ 0.4999981   0.4000017 ]]
Biases:
[5.1558018e-06 2.9999882e-01]
Layer 2:
Weights:
[[ 0.4999976  -0.59999734]
 [ 0.6000049   0.7000043 ]]
Biases:
[0.2999985  0.19999862]

Expected final weights:
Layer 1:
Weights:
[[0.08592, 0.32276], [-0.33872, 0.46172], [0.449984, 0.440072]]
Layer 2:
Weights:
[[0.2748, 0.188], [0.435904, -0.53168], [0.68504, 0.7824]]
Test Loss: [0.11265294998884201, 1.0]
Accuracy: 1.0
